In [1]:
import sys
import os
from git_root import git_root

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.optimizers import Adam

sys.path.append(git_root("utils"))
from utils import load_params, load_config

sys.path.append(git_root("training"))
from fetch_data import fetch_data_local, fetch_format_stacked
from fetch_data import to_numpy_arrays, prepare_tf_dataset

print(tf.__version__)

2.0.0


# Generating the predictions for the test sets

In [2]:
params = load_params()

angles = params["GLCM"]["mel_map"]["angles_in_deg"]

test = {
    "spectrogram": {0: []},
    "mel_map": {
        angle: [] for angle in angles
    }
}

In [3]:
from tensorflow.keras.models import load_model
sys.path.append(git_root("models"))
from affine_scalar_layer import AffineScalar


mapping_for_saved_models = {'spectrogram': 'spec',
                           'mel_map': 'mel'}

for map_type in test:
    for angle in test[map_type]:
        #Fetch the data
        print(f"Map: {map_type}, angle: {angle}")
        df = fetch_data_local(
            map_type=map_type, train=False, angle=angle
        )
        #Fetch the trained model
        model_path = git_root('models','saved_models',
                                  'glcm_{}_angle{}.h5'.format(mapping_for_saved_models[map_type],angle))
        net = load_model(model_path,  custom_objects={'AffineScalar':AffineScalar()})
        #For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
        dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(to_numpy_arrays(df)[0], tf.float32))
        #Generate predictions
        y = net.predict(dataset)
        preds = pd.DataFrame(y)
        preds.columns = ["class{}".format(i) for i in range(10)]
        
        #Concatenate the prediction to our dataset
        df = df.merge(preds, left_index=True, right_index=True)
        df = df.drop(['maps'], axis=1)
        
        save_path=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        print("Saving to: {}".format(save_path))
        df.to_csv(save_path)

Map: spectrogram, angle: 0
Fetching: data_spectrogram_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_spectrogram_0.csv
Map: mel_map, angle: 0
Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_0.csv
Map: mel_map, angle: 45
Fetching: data_mel_map_angle_45_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_45.csv
Map: mel_map, angle: 90
Fetching: data_mel_map_angle_90_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_90.csv
Map: mel_map, angle: 135
Fetching: data_mel_map_angle_135_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_135.csv


In [4]:
fetch_data_local(
            map_type='mel_map', train=False, angle=angle
        )

Fetching: data_mel_map_angle_135_test.json


,filename,maps,split_id,genre
0,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",0,blues
1,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",1,blues
2,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",2,blues
3,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",3,blues
4,blues.00018.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",4,blues
...,...,...,...,...
1395,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",9,rock
1396,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",10,rock
1397,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",11,rock
1398,rock.00096.wav,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",12,rock


In [7]:
temp = []

for i in range(len(angles)):
    temp.append(fetch_data_local(
            map_type='mel_map', train=False, angle=angles[i]
        )['maps'])

stacked_test = (
    [
        np.stack([
            np.squeeze(temp[j][i]) for j in range(len(angles))
        ], -1) for i in range(1400)
    ]
)

Fetching: data_mel_map_angle_0_test.json
Fetching: data_mel_map_angle_45_test.json
Fetching: data_mel_map_angle_90_test.json
Fetching: data_mel_map_angle_135_test.json


In [8]:
dataset = tf.data.Dataset.from_tensor_slices(tf.dtypes.cast(stacked_test, tf.float32)).batch(32)
dataset

<BatchDataset shapes: (None, 15, 15, 4), types: tf.float32>

In [9]:
#For stacked model

#Fetch the trained model
model_path = git_root('models','saved_models',
                                  'glcm_{}_stacked.h5'.format(mapping_for_saved_models[map_type],angle))
net = load_model(model_path,  custom_objects={'AffineScalar':AffineScalar()})
#For some reason, I need to cast my tensor to float32 for predict (and not for fit ?!)
#dataset = tf.data.Dataset.from_tensors(tf.dtypes.cast(fetch_format_stacked(train=False), tf.float32))
dataset = tf.data.Dataset.from_tensor_slices(tf.dtypes.cast(stacked_test, tf.float32)).batch(32)
#Generate predictions
y = net.predict(dataset)
preds = pd.DataFrame(y)
preds.columns = ["class{}".format(i) for i in range(10)]
        
#Concatenate the prediction to our dataset
df = fetch_data_local(map_type='mel_map', train=False, angle='0').loc[:,['filename','split_id','genre']].merge(preds,left_index=True, right_index=True)
        
save_path=git_root("evaluation","evaluation_results","results_{}_{}.csv".format('mel_map','stacked'))
print("Saving to: {}".format(save_path))
df.to_csv(save_path)

Fetching: data_mel_map_angle_0_test.json
Saving to: /Users/arnaudstiegler/Desktop/deep-music-classification/evaluation/evaluation_results/results_mel_map_stacked.csv


# Results

In [10]:
config = load_config()

label_names = config["genres"]
label_to_idx = dict((name, index) for index, name in enumerate(label_names))

class_columns = ["class{}".format(i) for i in range(10)]

**Per Piece**

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

piece_level_acc = {'spectrogram':{},
                      'mel_map': {}}

test['mel_map']['stacked'] = {}

for map_type in test:
    for angle in test[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        df['pred'] = np.argmax(df[class_columns].values, axis=1)
        df['label'] = df['genre'].apply(lambda x: label_to_idx[x])
        piece_level_acc[map_type][angle] = accuracy_score(df['label'], df['pred'])
        
        print(classification_report(df['label'], df['pred']))

              precision    recall  f1-score   support

           0       0.50      0.19      0.28       140
           1       0.84      0.89      0.86       140
           2       0.46      0.24      0.31       140
           3       0.34      0.26      0.30       140
           4       0.26      0.11      0.15       140
           5       0.46      0.59      0.52       140
           6       0.79      0.88      0.83       140
           7       0.37      0.81      0.51       140
           8       0.41      0.58      0.48       140
           9       0.12      0.11      0.11       140

    accuracy                           0.46      1400
   macro avg       0.46      0.46      0.44      1400
weighted avg       0.46      0.46      0.44      1400

              precision    recall  f1-score   support

           0       0.62      0.37      0.46       140
           1       0.73      0.77      0.75       140
           2       0.37      0.31      0.34       140
           3       0.35 

In [12]:
piece_level_acc

{'spectrogram': {0: 0.4642857142857143},
 'mel_map': {0: 0.43785714285714283,
  45: 0.40214285714285714,
  90: 0.38357142857142856,
  135: 0.40285714285714286,
  'stacked': 0.39357142857142857}}

**Per track**

In [13]:
#Generate a table that contains mapping (track: genre)
track_mapping = df[['filename','genre']].drop_duplicates().reset_index(drop=True)

In [14]:
track_level_acc = {'spectrogram':{},
                      'mel_map': {}}

for map_type in test:
    for angle in test[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        df_per_track = df.groupby('filename').sum()
        df_per_track['pred'] = np.argmax(df_per_track[class_columns].values, axis=1)
        df_result = df_per_track.merge(track_mapping, on='filename')
        df_result['label'] = df_result['genre'].apply(lambda x: label_to_idx[x])
        track_level_acc[map_type][angle] = accuracy_score(df_result['label'], df_result['pred'])

print(track_level_acc)

{'spectrogram': {0: 0.5}, 'mel_map': {0: 0.52, 45: 0.42, 90: 0.41, 135: 0.46, 'stacked': 0.43}}


**Per genre**

In [60]:
genre_level_acc = {'spectrogram':{},
                      'mel_map': {}}

for map_type in test:
    for angle in test[map_type]:
        filepath=git_root("evaluation","evaluation_results","results_{}_{}.csv".format(map_type,angle))
        df = pd.read_csv(filepath, index_col=0)
        df_per_track = df.groupby('filename').sum()
        df_per_track['pred'] = np.argmax(df_per_track[class_columns].values, axis=1)
        df_result = df_per_track.merge(track_mapping, on='filename')
        df_result['label'] = df_result['genre'].apply(lambda x: label_to_idx[x])
        track_level_acc[map_type][angle] = accuracy_score(df_result['label'], df_result['pred'])


KeyError: 'filename'